<a href="https://colab.research.google.com/github/CalebEng/Ai-Project/blob/master/Ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import torch
from torch import nn

!pip install --upgrade tiktoken
import tiktoken
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Ai work')
#torch.__version__
enc = tiktoken.encoding_for_model('gpt-3.5-turbo')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.1 MB/s eta 0:00:00
Mounted at /content/drive



------------------- Prep and loading data sets -------------------

--ONLY RUN THE NEXT IF CSV FILE CONTAINING CONVOS HAS A INDEX COLUMN--

In [ ]:
import pandas as pd
df = pd.read_csv('Conversation.csv')
# If you know the name of the column skip this
first_column = df.columns[0]
# Delete first
df = df.drop([first_column], axis=1)
df.to_csv('Conversation.csv', index=False)


In [ ]:
df.to_csv('Conversation_enc.csv',index=False)

In [ ]:
test = torch.tensor(temp1)

ValueError: ignored

In [ ]:
def format(x):
  y=enc.encode(x)
  return torch.tensor(y)

df['question'] = df['question'].map(format)
df['answer'] = df['answer'].map(format)


In [ ]:
df = pd.read_csv('Conversation.csv')

In [ ]:
df

In [ ]:

df2 = pd.read_csv('Conversation.csv')

questions = df2['question'].tolist()
answers = df2['answer'].tolist()

temp = map(format, questions)
temp2 = map(format, answers)

questions = list(temp)
answers = list(temp2)


--EXIT--

--Padding and turning data into tensors--

In [15]:
df = pd.read_csv('Conversation.csv')

temp1 = df['question'].tolist()
temp2 = df['answer'].tolist()

def format(x):
  y=enc.encode(x)
  return y

qT = list(map(format, temp1))
aT = list(map(format, temp2))

for i in range(0,len(qT)):
  if len(qT[i])<25:
    while len(qT[i])<25:
      qT[i].append(482)


for i in range(0,len(aT)):
  if len(aT[i])<25:
    while len(aT[i])<25:
      aT[i].append(482)





In [16]:

questions = torch.tensor(list(qT))
answers = torch.tensor(list(aT))

In [19]:
questions[0].type

<function Tensor.type>

TypeError: ignored

In [20]:
n = int(0.9*len(questions))

train_data_q = questions[:n]
train_data_a = answers[:n]

val_data_q = questions[n:]
val_data_a = answers[n:]

In [21]:
block_size = 8


In [22]:
x = train_data_q[:block_size]
y = train_data_a[:block_size]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target is: {target}")

when input is tensor([[6151,   11, 1268,  527,  499, 3815,   30,  482,  482,  482,  482,  482,
          482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,
          482]]) the target is: tensor([  72, 2846, 7060,   13, 1268,  922, 6261,   30,  482,  482,  482,  482,
         482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,
         482])
when input is tensor([[6151,   11, 1268,  527,  499, 3815,   30,  482,  482,  482,  482,  482,
          482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,
          482],
        [  72, 2846, 7060,   13, 1268,  922, 6261,   30,  482,  482,  482,  482,
          482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,  482,
          482]]) the target is: tensor([   72,  2846,  5128,  1695,    13,  9523,   369, 10371,    13,   482,
          482,   482,   482,   482,   482,   482,   482,   482,   482,   482,
          482,   482,   482,   482,   482])
when input is tensor([[ 6151,  

In [ ]:
batch_size = 4
block_size = 8

